In [1]:
import pandas as pd
import json
import re
import numpy as np

from sklearn.externals import joblib
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
pd.set_option('display.max_columns', 150)

## Clean data

In [3]:
path = '/Users/yuwenwu/insight/cultivate/data/processed/'
sentences_raw = pd.read_csv(path + 'enron_case_study_messages.csv')
recipients_raw = pd.read_csv(path + 'enron_case_study_chats.csv')

In [4]:
sentences_raw.head()

,sentence_index,sentiment_value,message__id,message__chat__id,message__sent_at,message__contact__parent__id,concreteness,request_prob,politeness,is_question,message__provider_guid,message__chat__team__id,Inform_prob,Conventional_prob,Request-Action_prob,Request-Information_prob,da_label,da_prob,msg_size,advs,all_lemmas,articles,aux_verbs,convo_index,convo_size,imp_prons,non_stop_word_lemmas,num_female,num_female_pronouns,num_male,num_male_pronouns,num_subordinate,num_superior,num_unique_tokens,num_unknowngender,num_unknownpower,parent_id,parent_prob,per_prons,preps,quants,response_time,sender_gender,sub_conjs
0,3,1,5kT1sN1BY08ZfoTYcUHo_Q,0EUsnl18IABPqYfgDRHaoA,2001-10-05 02:38:26+00:00,bM1QDebT6vOFy4ifK_OcGA,3.980000,0.154175,0.447116,False,723973,XZwXgkxlOGssBijnWEMOig,0.269913,0.122803,0.005557,0.006111,Inform,0.269913,3,0,parsing#file#--#>>#o#:#\#portland#\#westdesk#\...,0,0,0,1,0,parsing#file#portland#westdesk#california#sche...,0,0,5,0,0,0,21,6,11,NaN,NaN,0,0,0,NaN,I,0
1,2,2,5kT1sN1BY08ZfoTYcUHo_Q,0EUsnl18IABPqYfgDRHaoA,2001-10-05 02:38:26+00:00,bM1QDebT6vOFy4ifK_OcGA,4.465000,0.229237,0.433185,False,723973,XZwXgkxlOGssBijnWEMOig,0.102239,0.345797,0.004580,0.009907,Conventional,0.345797,3,0,log#message,0,0,0,1,0,log#message,0,0,5,0,0,0,2,6,11,NaN,NaN,0,0,0,NaN,I,0
2,1,2,5kT1sN1BY08ZfoTYcUHo_Q,0EUsnl18IABPqYfgDRHaoA,2001-10-05 02:38:26+00:00,bM1QDebT6vOFy4ifK_OcGA,2.160000,0.148573,0.399924,False,723973,XZwXgkxlOGssBijnWEMOig,0.309767,0.073297,0.006243,0.005516,Inform,0.309767,3,0,no#variance#detect#.,0,0,0,1,0,variance#detect,0,0,5,0,0,0,4,6,11,NaN,NaN,0,0,0,NaN,I,0
3,0,1,5kT1sN1BY08ZfoTYcUHo_Q,0EUsnl18IABPqYfgDRHaoA,2001-10-05 02:38:26+00:00,bM1QDebT6vOFy4ifK_OcGA,3.296667,0.047411,0.389760,False,723973,XZwXgkxlOGssBijnWEMOig,0.357122,0.000236,0.006312,0.002823,Inform,0.357122,3,0,start#date#:#NUMBER/NUMBER/NUMBER#;#hourahead#...,0,0,0,1,0,start#date#hourahead#hour#ancillary#schedule#a...,0,0,5,0,0,0,15,6,11,NaN,NaN,0,0,0,NaN,I,0
4,3,1,5NLpNTSV2Ui1EZdCWFTbAg,0EUsnl18IABPqYfgDRHaoA,2001-10-06 07:38:26+00:00,bM1QDebT6vOFy4ifK_OcGA,3.980000,0.154175,0.447116,False,724694,XZwXgkxlOGssBijnWEMOig,0.269913,0.122803,0.005557,0.006111,Inform,0.269913,3,0,parsing#file#--#>>#o#:#\#portland#\#westdesk#\...,0,0,0,1,0,parsing#file#portland#westdesk#california#sche...,0,0,5,0,0,0,21,6,11,NaN,NaN,0,0,0,NaN,I,0


In [108]:
msg_info = sentences_raw.filter(['message__provider_guid',
                      'message__chat__id', 
                      'message__sent_at', 
                      'message__contact__parent__id'])

msg_info.columns = ['msg_id', 'thread_id', 'timestamp', 'sender_id']
msg_info['timestamp'] = pd.to_datetime(msg_info['timestamp'])
msg_info = msg_info.drop_duplicates(keep = False)

In [ ]:
msg_info.to_csv('msg_sender_info.csv', index = False)

Recipient information is listed as multiple recipients per thread, with each recipient separated by a `#`. Split these strings and list each recipient individually along with their associated thread ID.

In [109]:
receive_info = recipients_raw[['chat_id', 'members']]
receive_info.columns = ['thread_id', 'member']

In [110]:
members = receive_info['member'].tolist()
split_members = []
for member in members:
    list_members = member.split('#')
    split_members.append(list_members)

In [111]:
len(split_members) == len(receive_info)

True

In [112]:
threads = receive_info['thread_id'].tolist()
map_thread_mem = {}
for thread, member in zip(threads, split_members):
    for m in member:
        map_thread_mem[m] = thread

In [113]:
receive_info = pd.DataFrame({
    'thread_id':list(map_thread_mem.values()),
    'receiver_id':list(map_thread_mem.keys())
})

In [114]:
receive_info = receive_info[receive_info.columns[::-1]]
receive_info.head()

,thread_id,receiver_id
0,zkUHTcKYUI7_3XtsPgH1AQ,301LjU_cYSyatHsS_QqARA
1,zxyV8XxngsP7JilFp3eQqA,70_DUyZ72y5ORTmgypWxcQ
2,yLnmZ6k8UCwav3DYKYLjhg,DCYh6--VXVhsATQEoc4j-w
3,zfB8SPtC--jfcpd51KpqMQ,NSgtxt71Wvio-rwUfUlsWQ
4,zfB8SPtC--jfcpd51KpqMQ,UOIL56jh0lDJaM5JXH6YBg


In [145]:
receive_info.to_csv('receive_info.csv', index = False)

Some threads do not have sender information, while others do not have recipient information (see below for an example). Remove threads that are not cross-listed. Thread IDs must be used because the recipient information only contains the thread ID, not the message ID.

In [115]:
t = 'zfB8SPtC--jfcpd51KpqMQ'
msg_info[msg_info['thread_id'] == t]
receive_info[receive_info['thread_id'] == t]

,msg_id,thread_id,timestamp,sender_id


,thread_id,receiver_id
3,zfB8SPtC--jfcpd51KpqMQ,NSgtxt71Wvio-rwUfUlsWQ
4,zfB8SPtC--jfcpd51KpqMQ,UOIL56jh0lDJaM5JXH6YBg
10,zfB8SPtC--jfcpd51KpqMQ,po966s5Lz0cydUg7lguFhg
1032,zfB8SPtC--jfcpd51KpqMQ,EZxwf9MgK3f-WHPxSQ5e0Q
1049,zfB8SPtC--jfcpd51KpqMQ,Wj0xuuVlQezwrqxLsFYCgw
4344,zfB8SPtC--jfcpd51KpqMQ,PsLcy-3e6IuETacYGV9a6Q
8486,zfB8SPtC--jfcpd51KpqMQ,IKfJz7SsfW4BqcIOqI1mCg


In [69]:
sender_unique_thread = msg_info['thread_id'].unique().tolist()
receive_unique_thread = receive_info['thread_id'].unique().tolist()

Intersection of thread IDs.

In [70]:
unique_threads = list(set(sender_unique_thread) & set(receive_unique_thread))

In [71]:
msg_info_unique = msg_info[msg_info['thread_id'].isin(unique_threads)]
receive_info_unique = receive_info[receive_info['thread_id'].isin(unique_threads)]

In [76]:
sorted_msg_date = msg_info_unique.sort_values('timestamp', ascending = True)
sorted_msg_date.sample(5)

,msg_id,thread_id,timestamp,sender_id
446820,825287,uR34AKaAh21RJYY9-EklLQ,2000-02-18 03:41:00,h5qfWQ06WAOMlNt-FdP2DA
19759,276447,-gFBe1bWCE7EMKeYHdnugA,2001-04-29 18:38:16,gX4EgYNeE8aZY0mLE3ZUMA
402758,742197,pdyLaEM6PAUsmLjLVkIwZw,2001-10-17 18:09:26,mNMbGJ7VAJgjntvFKEIo9A
404159,436353,q02VRwKYRvzwipFoP9bPRQ,2001-05-14 14:06:00,bj9PrVnHTyZ1ingaGWI_Bg
214040,1127089,RmHaBEYaBjvykXyT5KaUgg,2001-09-24 18:15:48,2BKdGZ1bkdkutupLevnzzw


In [90]:
single_msg = 0
threads = 0
send = []
receive = []
for thread in sorted_msg_date['thread_id'].unique():
    sender_df = sorted_msg_date[sorted_msg_date['thread_id'] == thread]
    receive_df = receive_info[receive_info['thread_id'] == thread]
    if len(sender_df) == 1:
        single_msg += 1
    else:
        threads += 1
        send.append(sender_df)
        receive.append(receive_df)

single_msg, threads

(1133, 136)

In [95]:
i = 3
send[i]
receive[i]

,msg_id,thread_id,timestamp,sender_id
175597,996340,MrmYbnxzrYrweb6hqJ-14w,2000-03-22 07:34:00,gsOSqVrCPhfwrbYM5C-faQ
175744,996333,MrmYbnxzrYrweb6hqJ-14w,2000-03-22 14:33:00,67BuRFiI3-WM5m7LRJXLKg
175750,996332,MrmYbnxzrYrweb6hqJ-14w,2000-03-22 14:38:00,67BuRFiI3-WM5m7LRJXLKg
175633,996312,MrmYbnxzrYrweb6hqJ-14w,2000-03-24 10:10:00,isqKRlccnbrulUTVLGgSoQ


,thread_id,receiver_id
34350,MrmYbnxzrYrweb6hqJ-14w,dtcKm7kungo6DQ97MeIIQg
46615,MrmYbnxzrYrweb6hqJ-14w,0TO5E7Z7wQIltTjzFC3eRw
46616,MrmYbnxzrYrweb6hqJ-14w,1RTmhbJD2Y72Jd3BEOoDog
46617,MrmYbnxzrYrweb6hqJ-14w,1tusYIDwqcA8bZbMbc0Srw
46618,MrmYbnxzrYrweb6hqJ-14w,2dJOigaWxDYHwkascdPxCA
46619,MrmYbnxzrYrweb6hqJ-14w,4A7m5Y4Td0PnFAeJPQP7Vg
46620,MrmYbnxzrYrweb6hqJ-14w,4b83JA_HhaIsQuNuigCi9A
46621,MrmYbnxzrYrweb6hqJ-14w,4ttbJuTLbm7ag0yBEcYo6w
46622,MrmYbnxzrYrweb6hqJ-14w,67BuRFiI3-WM5m7LRJXLKg
46623,MrmYbnxzrYrweb6hqJ-14w,6FuIv3TSo5VfMXrebLRshQ


In [89]:
len(sorted_msg_date['thread_id'].unique())

1269

## Power dynamics

In [84]:
power_dynamics = pd.read_csv(path + 'enron_power_pairs.csv')
power_dynamics.head()

,subordinate_uid,subordinate_contact_id,superior_uid,superior_contact_id
0,701,yYqeskxwbaauIUG4KECXew,40164,nvhoW44QWGidqYYAM7QMmw
1,701,yYqeskxwbaauIUG4KECXew,20805,DksWJ6nb6ZbWZb8TCyZF7g
2,19874,mKSspOBcszjlkrA9AJekFA,40164,nvhoW44QWGidqYYAM7QMmw
3,19874,mKSspOBcszjlkrA9AJekFA,27104,lCcz6Ffc3HxacQ7mYxZmhA
4,19874,mKSspOBcszjlkrA9AJekFA,20805,DksWJ6nb6ZbWZb8TCyZF7g


In [97]:
power_dynamics['subordinate_contact_id'].nunique()
power_dynamics['superior_contact_id'].nunique()
len(power_dynamics)

1501

299

11839

In [135]:
subord_id = []
for sender in msg_info['sender_id'].tolist():
    if sender in power_dynamics['subordinate_contact_id'].unique().tolist():
        subord_id.append(sender)
    else:
        subord_id.append(np.nan)

superior_id = []
for sender in msg_info['sender_id'].tolist():
    if sender in power_dynamics['superior_contact_id'].unique().tolist():
        superior_id.append(sender)
    else:
        superior_id.append(np.nan)
        
msg_info['sender_superior'] = superior_id
msg_info['sender_subord'] = subord_id

In [138]:
msg_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12045 entries, 17 to 490574
Data columns (total 6 columns):
msg_id             12045 non-null int64
thread_id          12045 non-null object
timestamp          12045 non-null datetime64[ns]
sender_id          12045 non-null object
sender_superior    2351 non-null object
sender_subord      5433 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 978.7+ KB


# SQL

In [150]:
import sqlite3
import pandas as pd

db_path = '/Users/yuwenwu/insight/cultivate/'
db = sqlite3.connect(db_path + 'enron_cultivate.db')

def run_query(query, db = db):
    return pd.read_sql_query(query, db)

Select subordinate individuals from enron_power_pairs who sent an email to a superior.

SELECT orders.order_number, customers.name
FROM orders
INNER JOIN customers
ON orders.customer_id = customers.customer_id

In [166]:
#selects subordinate sender emailing superior
query = '''
SELECT DISTINCT msg.msg_id as message, msg.timestamp as timestamp
FROM msg_sender_info AS msg
JOIN enron_power_pairs AS pp
ON pp.subordinate_contact_id = msg.sender_id
WHERE msg.thread_id IN (SELECT r.thread_id
FROM receive_info AS r
JOIN enron_power_pairs AS pp
ON pp.superior_contact_id = r.receiver_id)
'''

run_query(query)

,message,timestamp
0,178074,2001-03-21 03:38:00
1,877323,2001-10-25 13:19:52
2,904371,2000-08-23 07:36:00
3,1293640,2001-11-07 09:52:55
4,1351546,2001-11-15 18:04:41
5,286365,2001-05-06 16:16:22
6,1029645,2001-05-15 11:53:00
7,1278504,2001-11-07 00:03:41
8,1308487,2001-11-20 12:28:52
9,1293342,2001-11-21 15:43:02


In [167]:
#selects superior emailing inferior
query = '''
SELECT DISTINCT msg.msg_id as message, msg.timestamp as timestamp
FROM msg_sender_info AS msg
JOIN enron_power_pairs AS pp
ON pp.superior_contact_id = msg.sender_id
WHERE msg.thread_id IN (SELECT r.thread_id
FROM receive_info AS r
JOIN enron_power_pairs AS pp
ON pp.subordinate_contact_id = r.receiver_id)
'''

run_query(query)

,message,timestamp
0,1293528,2001-11-06 17:47:57
1,1087223,2001-01-19 03:30:00
2,1124649,2001-09-28 16:48:46
3,1250426,2002-02-05 10:11:57
4,877323,2001-10-25 13:19:52
5,847057,2000-06-09 10:04:00
6,561308,1979-12-31 19:00:00
7,904371,2000-08-23 07:36:00
8,1293640,2001-11-07 09:52:55
9,220477,1979-12-31 19:00:00


In [167]:
#select 
query = '''
SELECT DISTINCT msg.msg_id as message, msg.timestamp as timestamp
FROM msg_sender_info AS msg
JOIN enron_power_pairs AS pp
ON pp.superior_contact_id = msg.sender_id
WHERE msg.thread_id IN (SELECT r.thread_id
FROM receive_info AS r
JOIN enron_power_pairs AS pp
ON pp.subordinate_contact_id = r.receiver_id)
'''

run_query(query)

,message,timestamp
0,1293528,2001-11-06 17:47:57
1,1087223,2001-01-19 03:30:00
2,1124649,2001-09-28 16:48:46
3,1250426,2002-02-05 10:11:57
4,877323,2001-10-25 13:19:52
5,847057,2000-06-09 10:04:00
6,561308,1979-12-31 19:00:00
7,904371,2000-08-23 07:36:00
8,1293640,2001-11-07 09:52:55
9,220477,1979-12-31 19:00:00
